In [1]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, FloatType
from snowflake.snowpark.functions import avg, sum, col, udf, call_udf, call_builtin, year,is_null
import pandas as pd
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL


In [2]:
engine = create_engine(URL(
    account='UCLSGAD-LY00738',
    user = 'sherif1922',
    password = 'Mohamed1922',
    database = 'MOVIES',
    schema = 'public',
    warehouse = 'COMPUTE_WH',
    role='ACCOUNTADMIN',
))





In [3]:

connection = engine.connect()
results = connection.execute('select current_version()').fetchone()
print(results[0])


7.16.1


In [4]:

# read tmdb-movies data
df =  pd.read_csv('tmdb-movies.csv')
#specify the columns that I will use in the analysis process
movies_df = df[['original_title','genres','director','popularity','vote_average','budget_adj','revenue_adj','release_year']]
connection.execute("USE MOVIES")
#cur.execute("USE SCHEMA PUPLIC")

In [5]:

# lading data to statging layer on snowflake
if_exists = 'replace'
table_name = 'movies'
movies_df.to_sql(name=table_name.lower(), con=connection, index=False ,if_exists=if_exists)



10866

In [6]:
connection_parameters = {
   "account": "UCLSGAD-LY00738",
   "user": "sherif1922",
   "password": "Mohamed1922",
   "warehouse": "compute_wh",
   "role": "accountadmin",
   "database": "MOVIES",
   "schema": "PUBLIC"
}
session = Session.builder.configs(connection_parameters).create()

In [7]:
current_db = session.get_current_database()
print(current_db)

"MOVIES"


In [11]:
movies_df = session.table('movies')

In [12]:
movies_df.describe().show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"ORIGINAL_TITLE"  |"GENRES"          |"DIRECTOR"                 |"POPULARITY"        |"VOTE_AVERAGE"      |"BUDGET_ADJ"       |"REVENUE_ADJ"       |"RELEASE_YEAR"      |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|count      |10866             |10843             |10822                      |10866.0             |10866.0             |10866.0            |10866.0             |10866.0             |
|mean       |NULL              |NULL              |NULL                       |0.6464409519602429  |5.9749217743419845  |17551039.82288678  |51364363.25325097   |2001.322658         |
|stddev     |NULL              |NULL              |NULL                       |1

In [13]:
movies_df = movies_df.dropDuplicates()

In [14]:
movies_df.describe().show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"ORIGINAL_TITLE"  |"GENRES"          |"DIRECTOR"                 |"POPULARITY"        |"VOTE_AVERAGE"      |"BUDGET_ADJ"        |"REVENUE_ADJ"       |"RELEASE_YEAR"     |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|count      |10865             |10842             |10821                      |10865.0             |10865.0             |10865.0             |10865.0             |10865.0            |
|mean       |NULL              |NULL              |NULL                       |0.6464455549010584  |5.97501150483203    |17549894.037320547  |51369001.75884262   |2001.321859        |
|stddev     |NULL              |NULL              |NULL                       |1

In [15]:
movies_df =movies_df.dropna()

In [16]:
movies_df.describe().show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"ORIGINAL_TITLE"  |"GENRES"          |"DIRECTOR"                 |"POPULARITY"        |"VOTE_AVERAGE"      |"BUDGET_ADJ"       |"REVENUE_ADJ"       |"RELEASE_YEAR"      |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|count      |10800             |10800             |10800                      |10800.0             |10800.0             |10800.0            |10800.0             |10800.0             |
|mean       |NULL              |NULL              |NULL                       |0.6494548775        |5.970879629629629   |17654371.81052634  |51677135.50526419   |2001.289537         |
|stddev     |NULL              |NULL              |NULL                       |1

In [17]:
popular_movies = movies_df[['original_title','popularity']].sort(col('popularity'),ascending = False)
popular_movies = popular_movies.limit(10)

In [20]:
popular_movies.write.mode("overwrite").save_as_table("popular_movies")

# #Top 10 rated movies

In [21]:
movies_rate = movies_df[['original_title','vote_average']].sort(col('vote_average'),ascending = False)
top_10_rated_movies = popular_movies.limit(10)

In [23]:
top_10_rated_movies.write.mode("overwrite").save_as_table("top_rated_movies")

# top ten profitable movies

In [28]:
movies_df = movies_df.with_column("profit", (movies_df["revenue_adj"] - movies_df["budget_adj"]))

In [30]:
profitble_movies = movies_df[['original_title','profit']].sort(col('profit'),ascending = False)
top_10_profitble_movies = profitble_movies.limit(10)

In [31]:
top_10_profitble_movies.write.mode("overwrite").save_as_table("PROFITABLE_MOVIES")